# CHAPTER 8 - What is Quantum Machine Learning? - Code

*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install tensorflow==2.9.1

In [ ]:
pip install scikit-learn==1.0.2

In [ ]:
pip install matplotlib==3.2.2

In [ ]:
import numpy as np
seed = 128
np.random.seed(seed)

In [ ]:
from sklearn.datasets import make_classification

data, labels = make_classification(n_samples = 2500, 
    n_features = 2, n_informative = 2, n_redundant = 0, 
    weights = (0.2,0.8), class_sep = 0.5, random_state = seed)

In [ ]:
import matplotlib.pyplot as plt

for i in range(2):
    plt.hist(data[:,i][labels == 1], bins=100, alpha=0.8, label = "Negative")
    plt.hist(data[:,i][labels == 0], bins=100, alpha=0.8, label = "Positive")
    plt.legend()
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Split into a training and a test dataset.
x_tr, x_test, y_tr, y_test = train_test_split(
    data, labels, shuffle = True, train_size = 0.8)
# Split the test dataset to get a validation one.
x_val, x_test, y_val, y_test = train_test_split(
    x_test, y_test, shuffle = True, train_size = 0.5)

In [ ]:
import tensorflow as tf
tf.random.set_seed(seed)

model = tf.keras.Sequential([
    tf.keras.layers.Input(2),
    tf.keras.layers.Dense(8, activation = "elu"),
    tf.keras.layers.Dense(16, activation = "elu"),
    tf.keras.layers.Dense(8, activation = "elu"),
    tf.keras.layers.Dense(1, activation = "sigmoid"),
])

In [ ]:
opt = tf.keras.optimizers.Adam()
lossf = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer = opt, loss = lossf)

In [ ]:
history = model.fit(x_tr, y_tr,
    validation_data = (x_val, y_val), epochs = 8,
    batch_size = None)

In [ ]:
early_stp = tf.keras.callbacks.EarlyStopping(
    monitor = "val_loss", patience = 3, min_delta = 0.001)

In [ ]:
output = model.predict(x_test)
result = (output > 0.5).astype(float)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(result, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true = y_test, y_pred = result)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true = y_test, y_pred = result))

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, output)
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],linestyle="--",color="black")
plt.xlabel("FPR"); plt.ylabel("TPR")
plt.show()

In [ ]:
from sklearn.metrics import auc
print(auc(fpr,tpr))